In [36]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import TensorDataset, RandomSampler, DataLoader
from torch.optim import AdamW
import torch
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.functional as F

In [7]:
from datasets import load_dataset
# legal judgment prediction
dataset = load_dataset("/KLAID/KLAID.py", 'ljp')


Found cached dataset klaid (C:/Users/victolee/.cache/huggingface/datasets/klaid/ljp/1.0.0/3555cf1de16633bfd57ee9b1e854f43f3544b2c235bfbf0cca33b5056612fbef)
100%|██████████| 1/1 [00:00<00:00, 142.93it/s]


In [3]:
data = pd.read_csv('label.csv').iloc[:, :3]

In [22]:
laws = [None] * len(dataset['train'])
for i, val in enumerate(dataset['train']):
    laws[i] = data.iloc[val['laws_service_id']]['laws']
train = dataset['train'].add_column("laws", laws)

In [27]:
model_name = 'beomi/KcELECTRA-base-v2022'
q_tokenizer = AutoTokenizer.from_pretrained(model_name)
p_tokenizer = AutoTokenizer.from_pretrained(model_name)

q_seqs = q_tokenizer(train['fact'], padding="max_length", truncation=True, return_tensors="pt")
p_seqs = p_tokenizer(train['laws'], padding="max_length", truncation=True, return_tensors="pt")

train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'], 
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])


In [33]:
q_model = AutoModel.from_pretrained(model_name)
p_model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at beomi/KcELECTRA-base-v2022 were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at beomi/KcELECTRA-base-v2022 were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense

In [54]:
args = {
    'random_seed': 42, # Random Seed
    'pretrained_model': 'beomi/KcELECTRA-base',  # Transformers PLM name
    'pretrained_tokenizer': '',  # Optional, Transformers Tokenizer Name. Overrides `pretrained_model`
    'batch_size': 32,
    'lr': 5e-6,  # Starting Learning Rate
    'epochs': 50,  # Max Epochs
    'max_length': 150,  # Max Length input size
    'num_train_epochs': 3,
    'adam_epsilon': 1e-08,
    'gradient_accumulation_steps': 1,
    'warmup_steps': 5,
    'weight_decay': 0.1
}

In [56]:
train_sampler = RandomSampler(train)
train_dataloader = DataLoader(train, sampler=train_sampler, batch_size=args['batch_size'])
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in p_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args['weight_decay']},
    {'params': [p for n, p in p_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    {'params': [p for n, p in q_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args['weight_decay']},
    {'params': [p for n, p in q_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
######
optimizer = AdamW(optimizer_grouped_parameters, lr=args['lr'], eps=args['adam_epsilon'])

# 총학습 길이를 계산해서 scheduler에 넣어주기
t_total = len(train_dataloader) // args['gradient_accumulation_steps'] * args['epochs']
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args['warmup_steps'], num_training_steps=t_total)

# 학습 초기, 초기화
# optimizer.zero_grad() -> 설정해준 파라미터만 초기화
# model.zero_grad() -> 모델의 전체 파라미터를 초기화
p_model.zero_grad()
q_model.zero_grad()
torch.cuda.empty_cache()

train_iterator = trange(int(args['num_train_epochs']), desc="Epoch")
batch_loss = 0

# 학습 루프 시작!
for _ in train_iterator:
    print(_)
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")
    # batch 단위로 꺼내오기
    for step, batch in enumerate(epoch_iterator):
        q_model.train()
        p_model.train()
        
        if torch.cuda.is_available():
            batch = tuple(t.cuda() for t in batch)

            p_inputs = {'input_ids': batch[0],
                        'attention_mask': batch[1],
                        'token_type_ids': batch[2]
                        }
            
            q_inputs = {'input_ids': batch[3],
                        'attention_mask': batch[4],
                        'token_type_ids': batch[5]}
            
            p_outputs = p_model(**p_inputs).pooler_output # pooler_output이 [CLS] 토큰의 임베딩 벡터
            q_outputs = q_model(**q_inputs).pooler_output

            # 유사도 구하기 (batch_size * embedding_dims) * (embedding_dims, batch_size)
            sim_scores = torch.matmul(q_outputs, torch.transpose(p_outputs, 0, 1))

            targets = torch.arange(0, args['per_device_train_batch_size']).long()
        if torch.cuda.is_available():
            targets = targets.to('cuda')

            # log softmax를 취하고 nll_loss를 계산
            sim_scores = F.log_softmax(sim_scores, dim=1)
            loss = F.nll_loss(sim_scores, targets)

            loss.backward()
            optimizer.step()
            scheduler.step()
            q_model.zero_grad()
            p_model.zero_grad()
            batch_loss += loss.detach().cpu().numpy()
            torch.cuda.empty_cache()

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

0


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]


AttributeError: 'str' object has no attribute 'cuda'

In [60]:
[i for i in batch]

['laws_service_id', 'fact', 'laws_service', 'laws']

In [24]:

train[0]

{'laws_service_id': 32,
 'fact': '피고인은 2018. 8. 9. 23:33경 술을 마신 상태로 경산시 사동에 있는 상호 불상의 식당에서부터 같은 동에 있는 부영5차 앞 삼거리까지 B 스타렉스 승용차를 운전한 다음 승용차 안에서 잠을 자던 중, 차량 운전자가 시동을 걸어 놓고 잠을 자고 있다는 112 신고를 받고 현장에 출동한 경산경찰서 C파출소 소속 경위 D으로부터 피고인의 입에서 술 냄새가 나고 보행이 비틀거리는 등 술에 취한 상태에서 운전하였다고 인정할 만한 상당한 이유가 있어 약 10분 동안 총 3회에 걸쳐 음주측정기에 입김을 불어 넣는 방법으로 음주측정에 응할 것을 요구받고도 정당한 사유 없이 이에 응하지 아니하였다.',
 'laws_service': '도로교통법 제148조의2 제2항,도로교통법 제44조 제2항',
 'laws': '도로교통법 제148조의2 제2항 술에 취한 상태에 있다고 인정할 만한 상당한 이유가 있는 사람으로서 도로교통법 제44조제2항에 따른 경찰공무원의 측정에 응하지 아니하는 사람(자동차등 또는 노면전차를 운전하는 사람으로 한정한다)은 1년 이상 5년 이하의 징역이나 500만원 이상 2천만원 이하의 벌금에 처한다. \\n 도로교통법 제44조 제2항  경찰공무원은 교통의 안전과 위험방지를 위하여 필요하다고 인정하거나 제1항을 위반하여 술에 취한 상태에서 자동차등, 노면전차 또는 자전거를 운전하였다고 인정할 만한 상당한 이유가 있는 경우에는 운전자가 술에 취하였는지를 호흡조사로 측정할 수 있다. 이 경우 운전자는 경찰공무원의 측정에 응하여야 한다.'}